In [49]:
import json
import numpy
import pandas
from datetime import datetime
from pprint import pprint
import re
pandas.options.display.max_colwidth = -1
print('done')

done


In [50]:
# runs in about 5 minutes
#with open('events.json','r') as f:
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
print('done')

ok
done


In [51]:
#allgames_df.sample(10).index
#mygames = ['NYN201204100']
mygames = ['BAL201208060']
#mygames += ['SEA201208150']

#mygames = ['NY1193809221']

#mygames += ['BAL201208300', 'BAL201209300', 'BAL201304210']
#mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

somecolumns = allgames_df.columns
allgames2_df = pandas.DataFrame(columns=somecolumns)
allgames2_df['id'] = ''
allgames2_df = allgames2_df.set_index('id')
allgames2_df
for thisid, thisrow in allgames_df.iterrows():
    if (thisid in mygames):
        allgames2_df = allgames2_df.append(thisrow)
allgames_df = allgames2_df
print('done')

done


In [52]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

display_order = ['gameID', 'visiting_team', 'inning', 'batting_team']
display_order += ['outs', 'balls', 'strikes', 'pitch_sequence']
display_order += ['visitor_score', 'home_score', 'batterID', 'batter_hand', 'pitcherID', 'pitcher_hand']
#display_order += ['catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID']
#display_order += ['runner1b', 'runner2b', 'runner3b']
print('ok')

ok


In [53]:
def load_rosters():
    all_rosters_df = pandas.read_csv('fullroster.csv')
    return all_rosters_df

def get_starters(onerow):
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df

def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')
    df['baserunning'] = df['event'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df

def get_game_info(onerow, theindex):

    info_df = pandas.DataFrame.from_records(onerow['info'], index=theindex)

    teams_df = pandas.read_csv('teams.csv')
       
    # League is "A" for AL and "N" for NL
    info_df['visleague'] = teams_df['league'][teams_df['teamID'] == info_df['visteam'].tolist()[0]].tolist()[0][0:1]
    info_df['homeleague'] = teams_df['league'][teams_df['teamID'] == info_df['hometeam'].tolist()[0]].tolist()[0][0:1]

    info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
    info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
    info_df['dayofweek'] = info_df['date'].apply(lambda x: datetime.strftime(x, '%a'))
    
    info_df['gameno'] = pandas.to_numeric(info_df['number'], errors='coerce')
    info_df = info_df.drop('number', axis=1)
    info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
    info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
    info_df['timeofgame'] = pandas.to_numeric(info_df['timeofgame'], errors='coerce')
#    info_df['timeofgame'] = pandas.to_timedelta(info_df['timeofgame'].astype(str) + ' minutes', unit='m', errors='coerce')
    info_df = info_df.fillna('')

    return info_df

def get_inning_info(thisevent, infodf, inningno):
    
    thisevent['visiting_team'] = infodf['visteam'].tolist()[0]
    thisevent['home_team'] = infodf['hometeam'].tolist()[0]
    thisevent['inning'] = inningno
    if (side == 0):
        thisevent['batting_team'] = thisevent['visiting_team']
        thisevent['fielding_team'] = thisevent['home_team']
    else:
        thisevent['batting_team'] = thisevent['home_team']
        thisevent['fielding_team'] = thisevent['visiting_team']
    return thisevent

   
def get_fielders(thisevent, playing_df, fielding_flag):
                    
    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 1)].tolist()[0]                               

    thisevent['pitcher_hand'] = roster_df['throwing_hand'][
            (roster_df['playerID'] == thisevent['pitcherID']) & 
            (roster_df['year'] == float(thisevent['gameID'][3:7])) 
            ].tolist()[0]
    
    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 2)].tolist()[0]
    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 3)].tolist()[0]
    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 4)].tolist()[0]
    thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 5)].tolist()[0]
    thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 6)].tolist()[0]
    thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 7)].tolist()[0]
    thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 8)].tolist()[0]
    thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 9)].tolist()[0]

    return thisevent



def advance_runners(evnt, basers, oots, scr, teem):   # If there is baserunning info provided, obey it

#    print('\n')
#    print('ADVANCING RUNNERS...')
#    print('on base before: ',basers)
    
    # If there are multiple baserunning events, split them around ;
    x = evnt['baserunning'].split(';')

    success = []
    failure = []

    for i in range(0, len(x)):
        if ('-' in x[i]):
            success.append([x[i].split('-')[0], x[i].split('-')[1][0], x[i].split('-')[1][1:]])
        if ('X' in x[i]):
            failure.append([x[i].split('X')[0], x[i].split('X')[1][0], x[i].split('X')[1][1:]])

    for u in (success, failure):
        for j in range(0, len(u)):
            if (u[j][0] == 'B'):
                u[j][0] = 0
            if (u[j][1] == 'H'):
                u[j][1] = 4
            for k in range(0,2):
                u[j][k] = int(u[j][k])

#    print('Failure matrix: ', failure)
    for y in range(0, len(failure)):
        oots = oots + 1
#        print('Now {0:.0f} outs'.format(oots))
#        print('failure[y]: ',failure[y])
        basers[failure[y][0]-1] = ''
        basers[failure[y][1]-1] = ''
        evnt['comment'] += failure[y][2]     
#        print('Runner that was on base {0:.0f} is now out.'.format(success[y][0]))
        

#    print('Success matrix: ', success)
    for z in range(0, len(success)):
#        print('success[z]: ',success[z])
        if (success[z][1] == 4):
            scr[teem] = scr[teem] + 1
#            print('Score now {0:.0f} - {1:.0f}.'.format(scr[0], scr[1]))
        elif (success[z][0] == 0):
            basers[success[z][1]-1] = evnt['batterID']
        else:
            basers[success[z][1]-1] = basers[success[z][0]-1]

        basers[success[z][0]-1] = ''   # regardless, clear the base he was originally on
        evnt['comment'] += success[z][2]
#        print('Runner that was on base {0:.0f} is now on base {1:.0f}'.format(success[z][0], success[z][1]))

#        print('on base after: ',basers)
#        print('Score: ',scr)
        
    return (evnt, basers, oots, scr)

def advance_batter(evnt, basers, oots, scr, teem):   

#    print('\n')
#    print('ADVANCING BATTER...')
#    print('on base before: ', basers)
#    print('Batter: ', evnt['batterID'])
#    print('Play type: ', evnt['play'][0])


    # After baserunners have been advanced as needed, advance the batter too.
    if (evnt['play'][0] in ('W', 'I', 'F', 'E')):
        if ('B' not in evnt['baserunning']):   # now advance the batter to first, unless he's explicitly out
            basers[0] = evnt['batterID']

    elif (evnt['play'][0] == 'S'):
        if (evnt['play'][1] == 'B'):   # a single as opposed to a stolen base
            print('Stolen base!')
        elif ('B' not in evnt['baserunning']):   # now advance the batter to first, unless he's explicitly out
            basers[0] = evnt['batterID']
        else:
            pass   # we already explicitly advanced the batter above
            #print('basers: ',basers)
        
    elif (evnt['play'][0] == 'D'):
        if ('B' not in evnt['baserunning']):
                basers[1] = evnt['batterID']


    elif (evnt['play'][0] == 'T'):
        if ('B' not in evnt['baserunning']):
                basers[2] = evnt['batterID']

    elif (evnt['play'][0] == 'H'):
        if (evnt['play'][1] == 'R'):
            scr[teem] = scr[teem] + 1
            basers = ['', '', '']
        elif (evnt['play'][1] == 'P'):  # HP means hit by pitch
            basers[0] = evnt['batterID']
        else:
            print('H but not R or P')
            
    else:
        print('wtf starting letter not in set of {H|S|W|I|F|E}')


#    print('on base after: ',basers)
#    print('Score: ',scr)
#    print('---------------')

    return (evnt, basers, oots, scr)


def make_substitution(players_curr, evnt):
#    print('SUB at ev ',eig,': ', evnt)#evnt[['inning','home_or_visitor','sub_team','sub_batting_order','sub_position']].tolist())#thisevent[['batterID', 'sub_batting_order', 'sub_position']].tolist())
#    print('-------')
    if (evnt['home_or_visitor'] == evnt['sub_team']):
        print('batting sub')
        
        print('Old player: ',players_curr['playerID'].loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
        ].tolist()[0])
        
        players_curr.loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
        ] = evnt['batterID']
        
        print('New player: ', evnt['batterID'])

    else:
        print('fielding sub')
        
        print('Old player: ',players_curr['playerID'].loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
        ].tolist()[0])

        players_curr.loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
        ] = evnt['batterID']

        print('New player: ', evnt['batterID'])
        
#    print(playing_df.dtypes)
#    print(evnt[['home_or_visitor', 'sub_team', 'sub_position']])
#    print(playing_df[playing_df['which_team'] == evnt['sub_team']])
    return players_curr

print('done')

done


In [54]:
roster_df = load_rosters()

events_df = pandas.DataFrame()

for thisgameID,thisrow in allgames_df.iterrows():
    print(thisgameID+'...')
    game_events_df = pandas.DataFrame()
        
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    thisgame_df['gameID'] = thisgameID
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df['sub_team'] = pandas.to_numeric(thisgame_df['sub_team'])
    thisgame_df['sub_batting_order'] = pandas.to_numeric(thisgame_df['sub_batting_order'])
    thisgame_df['sub_position'] = pandas.to_numeric(thisgame_df['sub_position'])
    
    
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    this_info_df = get_game_info(thisrow, allgames_df.index) # get constants of game info
    playing_df = get_starters(thisrow) # get starters
    thisgame_df = split_event(thisgame_df) # split all events in game into parts    
    score = [0,0] # score as [vis, home] starts at [0,0]

    foundinnings = thisgame_df['inning'].drop_duplicates().values.tolist()
    theseinnings = numpy.arange(1, numpy.max(foundinnings)+2) # it's +2 b/c it expects a 0-based list

#    If we need to switch the sub side.
#    thisgame_df['home_or_visitor'] = numpy.where(thisgame_df['event_type'] == 'sub', 1 - thisgame_df['home_or_visitor'], thisgame_df['home_or_visitor'])
    
    for ing in theseinnings:
        for side in range(0,2):                           # iter through each half-inning
            halfinn = thisgame_df[(thisgame_df['inning'] == ing) & (thisgame_df['home_or_visitor'] == side)]
            onbase = ['', '', '']
            nOuts = 0            #     and there are no outs
            fieldingflag = 1 - side  # flag: makes fielding team the opposite of batting team

            for eig, thisevent in halfinn.iterrows():    # iter through all events in this half-inn    

                if (thisevent['event_type'] == 'sub'):
                    playing_df = make_substitution(playing_df, thisevent)
                else:
                    print(thisevent[['inning', 'home_or_visitor', 'event']].tolist())

                    thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
                    thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
                    thisevent['inning'] = ing
                    if (side == 0):
                        thisevent['batting_team'] = thisevent['visiting_team']
                        thisevent['fielding_team'] = thisevent['home_team']
                    else:
                        thisevent['batting_team'] = thisevent['home_team']
                        thisevent['fielding_team'] = thisevent['visiting_team']

                    thisevent = get_fielders(thisevent, playing_df, fieldingflag) # get fielders, including pitcher

                    thisevent['abflag'] = 1
                    thisevent['balls'] = pandas.to_numeric(thisevent['pitch_count'][0:1], errors='coerce')
                    thisevent['strikes'] = pandas.to_numeric(thisevent['pitch_count'][1:2], errors='coerce')

                    # batterID gets loaded from raw data, but look up his handendess... and....
                    # DON'T FORGET TO CORRECT THIS FOR BADJ IF NEEDED!
                    thisevent['batter_hand'] = roster_df['batting_hand'][(roster_df['playerID'] == thisevent['batterID']) & (roster_df['team'] == thisevent['batting_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]

#                    if (thisevent['play'] == 'NP'):
#                        print('\n')
#                        print('NO PLAY: ',eig, thisevent[['inning','home_or_visitor','batterID', 'event']].tolist())
#                        print('-------')

                    # if there is baserunning info, implement it first
                    if (thisevent['baserunning'] != ''):
                        (thisevent, onbase, nOuts, score) = advance_runners(
                            thisevent,
                            onbase,
                            nOuts,
                            score,
                            side)               

                    # If the play starts with fielder info or is a strikeout, increment outs
                    if (re.match(r'[0-9K]+',thisevent['play'])):
                        nOuts = nOuts + 1

                    # If the play is a hit, advance baserunners with fcn advance_runners(evnt, basers, oots, scr, teem):
                    if (re.match(r'[S|D|T|H|W|I|E|F]',thisevent['play'])):
                        (thisevent, onbase, nOuts, score) = advance_batter(
                            thisevent,
                            onbase,
                            nOuts,
                            score,
                            side)

                    # update info based on the play that has occurred
                    thisevent['outs'] = nOuts
                    thisevent['runner1b'] = onbase[0]
                    thisevent['runner2b'] = onbase[1]
                    thisevent['runner3b'] = onbase[2]
                    thisevent['visitor_score'] = score[0]
                    thisevent['home_score'] = score[0]

                # finally, append this play to the list of plays
                game_events_df = game_events_df.append(thisevent)
                
    game_events_df = game_events_df.sort_index()
    game_events_df.index.name = 'event_in_game'
    game_events_df = game_events_df.reset_index()

    events_df = pandas.concat((events_df, game_events_df), axis=0)

display_order = ['gameID', 'event_in_game', 'inning', 'batterID']
display_order += ['play', 'modifiers', 'baserunning', 'outs']
display_order += ['runner1b', 'runner2b', 'runner3b']

#events_df = events_df.fillna('')

events_df = events_df.reset_index()
events_df.index.name = 'eventID'

events_df.sort_values('gameID')
events_df = events_df[display_order]
print('Final score: ',score)
#events_df
#x = events_df['play'].tolist()
#for k in x:
#    print(k)

BAL201208060...
[1, 0, 'K23']
[1, 0, '8/F']
[1, 0, '13/G-']
[1, 1, 'S8/G']
[1, 1, 'K']
[1, 1, '2/P2F']
[1, 1, 'D8/L+.1-3']
[1, 1, '1/G-']
[2, 0, '8/F']
[2, 0, 'K']
[2, 0, '7/L']
[2, 1, 'K']
[2, 1, 'S7/L']
[2, 1, 'D8/L.1-H']
[2, 1, '7/F']
[2, 1, 'HR/9/F.2-H']
[2, 1, '63/G']
[3, 0, '43/G+']
[3, 0, '7/F']
[3, 0, '7/F']
[3, 1, '53/G']
[3, 1, '63/G']
[3, 1, '9/F']
[4, 0, '6/P']
[4, 0, 'W']
[4, 0, 'K']
[4, 0, '3/G']
[4, 1, '31/G']
[4, 1, '9/F']
[4, 1, 'S6/P']
[4, 1, '13/G-']
[5, 0, '8/F']
[5, 0, 'S7/G']
[5, 0, '46(1)/FO/G']
[5, 0, '5/P']
[5, 1, '43/G']
[5, 1, '63/G']
[5, 1, '63/G']
[6, 0, 'S4/BG.B-2(E4/TH1)']
[6, 0, '43/G.2-3']
[6, 0, '1!3/G+']
[6, 0, '9/F']
[6, 1, '63/G']
[6, 1, '63/G']
[6, 1, '43/G']
[7, 0, 'S9/F-']
[7, 0, 'K']
[7, 0, 'K']
[7, 0, '8/F']
[7, 1, '53/G-']
[7, 1, 'D7/L+']
[7, 1, '63/G']
[7, 1, '9/F']
[8, 0, 'NP']
fielding sub
Old player:  fordl001
New player:  mclon001
[8, 0, 'D9/L']


IndexError: list index out of range